In [1]:
import os
import shutil
import glob
import cv2
import numpy as np
from scipy.io import loadmat

print("Environment Ready")


Environment Ready


In [2]:
SRC = "/kaggle/input/shanghaitech/ShanghaiTech"
DST = "/kaggle/working/shanghaitech"

if not os.path.exists(DST):
    shutil.copytree(SRC, DST)

print("Dataset copied to:", DST)
print(os.listdir(DST))


Dataset copied to: /kaggle/working/shanghaitech
['part_B', 'part_A']


In [3]:
DATA_ROOT = "/kaggle/working/shanghaitech"

PART_A_TRAIN = f"{DATA_ROOT}/part_A/train_data"
PART_A_TEST  = f"{DATA_ROOT}/part_A/test_data"

print("Train data:", os.listdir(PART_A_TRAIN))
print("Test data:", os.listdir(PART_A_TEST))


Train data: ['images', 'ground-truth']
Test data: ['images', 'ground-truth']


In [4]:
def get_counts(folder):
    img_dir = os.path.join(folder, "images")
    gt_dir = os.path.join(folder, "ground-truth")

    img_paths = sorted(glob.glob(os.path.join(img_dir, "*.jpg")))
    counts = []

    for img_path in img_paths:
        base = os.path.basename(img_path).replace(".jpg", "")
        mat_path = os.path.join(gt_dir, f"GT_{base}.mat")

        mat = loadmat(mat_path)

        
        points = mat["image_info"][0][0][0][0][0]

        counts.append(len(points))

    return img_paths, np.array(counts)


In [5]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.feature import hog

def extract_features(img_paths, size=128):
    """
    Extract combined visual features for each image:
    - Resized grayscale pixels
    - HOG features
    - LBP features
    - Edge density
    - Brightness stats
    - ORB keypoint count
    """
    features = []

    # LBP params
    radius = 2
    n_points = 8 * radius

    for p in img_paths:
        img = cv2.imread(p)
        if img is None:
            print("Could not read:", p)
            continue
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_small = cv2.resize(gray, (size, size))

       
        #Flattened pixel intensities
        raw_pix = gray_small.flatten()

        # HOG descriptor 
        hog_feat = hog(
            gray_small,
            orientations=8,
            pixels_per_cell=(16, 16),
            cells_per_block=(1, 1),
            visualize=False,
            channel_axis=None
        )
        # LBP texture histogram
        lbp = local_binary_pattern(gray_small, n_points, radius, method="uniform")
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=int(lbp.max() + 1), density=True)

        # Edge Density (Canny)
        edges = cv2.Canny(gray_small, 100, 200)
        edge_density = np.sum(edges > 0) / edges.size  # 0–1

        # Brightness stats
        mean_intensity = np.mean(gray_small)
        var_intensity  = np.var(gray_small)

        # ORB keypoints count

        orb = cv2.ORB_create()
        keypoints = orb.detect(gray_small, None)
        kpt_count = len(keypoints)


        # Combine all features
        feat_vector = np.concatenate([
            raw_pix,           # pixel values
            hog_feat,          # structure
            lbp_hist,          # texture
            [edge_density],    # edges
            [mean_intensity, var_intensity],
            [kpt_count],       # keypoints
        ])

        features.append(feat_vector)

    return np.array(features)


In [6]:
from sklearn.model_selection import train_test_split


# Load image paths & GT counts
train_img_paths, train_counts = get_counts(PART_A_TRAIN)
test_img_paths,  test_counts  = get_counts(PART_A_TEST)

print("Train images:", len(train_img_paths), " Test images:", len(test_img_paths))


# Extract features
print("Extracting training features...")
train_features = extract_features(train_img_paths)

print("Extracting test features...")
test_features  = extract_features(test_img_paths)


# 3. Convert to X, y
train_X = train_features
train_y = train_counts

test_X  = test_features
test_y  = test_counts

print("Shapes:")
print("train_X:", train_X.shape)
print("train_y:", train_y.shape)
print("test_X:", test_X.shape)
print("test_y:", test_y.shape)


Train images: 300  Test images: 182
Extracting training features...
Extracting test features...
Shapes:
train_X: (300, 16918)
train_y: (300,)
test_X: (182, 16918)
test_y: (182,)


In [7]:
from sklearn.ensemble import RandomForestRegressor
import joblib

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

rf.fit(train_X, train_y)
print("Random Forest training completed!")

joblib.dump(rf, "/kaggle/working/random_forest_model.pkl")
print("Model saved as random_forest_model.pkl")


Random Forest training completed!
Model saved as random_forest_model.pkl


In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


# 1. Predict on train & test
pred_train = rf.predict(train_X)
pred_test  = rf.predict(test_X)


# Metrics
mae_train = mean_absolute_error(train_y, pred_train)
rmse_train = np.sqrt(mean_squared_error(train_y, pred_train))
r2_train = r2_score(train_y, pred_train)

mae_test = mean_absolute_error(test_y, pred_test)
rmse_test = np.sqrt(mean_squared_error(test_y, pred_test))
r2_test = r2_score(test_y, pred_test)


# Print summary

print("========== Random Forest Evaluation ==========")
print(f"Train MAE :  {mae_train:.3f}")
print(f"Train RMSE: {rmse_train:.3f}")
print(f"Train R²  : {r2_train:.3f}")
print("----------------------------------------------")
print(f"Test  MAE : {mae_test:.3f}")
print(f"Test  RMSE: {rmse_test:.3f}")
print(f"Test R²   : {r2_test:.3f}")
print("==============================================")


# 4. Show some predictions

for i in range(5):
    print(f"Image: {os.path.basename(test_img_paths[i])} | GT: {test_y[i]} | Pred: {pred_test[i]:.1f}")


========== Random Forest Evaluation ==========
Train MAE :  107.853
Train RMSE: 155.522
Train R²  : 0.905
----------------------------------------------
Test  MAE : 237.064
Test  RMSE: 296.678
Test R²   : 0.296
Image: IMG_1.jpg | GT: 172 | Pred: 575.9
Image: IMG_10.jpg | GT: 502 | Pred: 540.8
Image: IMG_100.jpg | GT: 389 | Pred: 463.0
Image: IMG_101.jpg | GT: 211 | Pred: 500.0
Image: IMG_102.jpg | GT: 223 | Pred: 549.4


In [9]:
import pandas as pd
import os

# -----------------------------
# 1. Save Metrics to CSV
# -----------------------------
metrics_df = pd.DataFrame({
    "Metric": ["Train MAE", "Train RMSE", "Train R²", "Test MAE", "Test RMSE", "Test R²"],
    "Value": [mae_train, rmse_train, r2_train, mae_test, rmse_test, r2_test]
})

metrics_df.to_csv("random_forest_metrics.csv", index=False)
print("Saved: random_forest_metrics.csv")

# -----------------------------
# 2. Save first 5 predictions to CSV
# -----------------------------
pred_df = pd.DataFrame({
    "Image": [os.path.basename(p) for p in test_img_paths[:5]],
    "Ground Truth": test_y[:5],
    "Prediction": pred_test[:5]
})

pred_df.to_csv("random_forest_predictions.csv", index=False)
print("Saved: random_forest_predictions.csv")


Saved: random_forest_metrics.csv
Saved: random_forest_predictions.csv


In [10]:
import matplotlib.pyplot as plt
import numpy as np

# -----------------------------------------
# 1. Scatter plot: Ground Truth vs Predicted Count
# -----------------------------------------
plt.figure(figsize=(6,6))
plt.scatter(test_y, pred_test, alpha=0.6)
plt.plot([test_y.min(), test_y.max()], 
         [test_y.min(), test_y.max()], 
         linestyle='--')
plt.xlabel("Ground Truth Count")
plt.ylabel("Predicted Count")
plt.title("GT vs Predicted (Scatter Plot)")
plt.grid(True)
plt.savefig("scatter_gt_pred.png", dpi=300, bbox_inches='tight')
plt.close()

# -----------------------------------------
# 2. Error Histogram
# -----------------------------------------
errors = pred_test - test_y

plt.figure(figsize=(8,5))
plt.hist(errors, bins=40)
plt.xlabel("Prediction Error (Pred - GT)")
plt.ylabel("Frequency")
plt.title("Error Distribution")
plt.grid(True)
plt.savefig("error_histogram.png", dpi=300, bbox_inches='tight')
plt.close()

# -----------------------------------------
# 3. Line Plot for first 50 images: GT vs Pred
# -----------------------------------------
N = min(50, len(test_y))
plt.figure(figsize=(12,5))
plt.plot(range(N), test_y[:N], label="Ground Truth")
plt.plot(range(N), pred_test[:N], label="Predicted")
plt.xlabel("Sample Index")
plt.ylabel("Crowd Count")
plt.title("GT vs Pred for First 50 Samples")
plt.legend()
plt.grid(True)
plt.savefig("lineplot_gt_pred_50.png", dpi=300, bbox_inches='tight')
plt.close()

# -----------------------------------------
# 4. Residual Plot
# -----------------------------------------
plt.figure(figsize=(8,5))
plt.scatter(test_y, errors, alpha=0.6)
plt.axhline(0, color='black', linestyle='--')
plt.xlabel("Ground Truth Count")
plt.ylabel("Residual (Pred - GT)")
plt.title("Residual Plot")
plt.grid(True)
plt.savefig("residual_plot.png", dpi=300, bbox_inches='tight')
plt.close()

print("All plots saved successfully as PNG files!")


All plots saved successfully as PNG files!


# CNN

### Simple CNN Model 

In [9]:
import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import pandas as pd


# ============================================================
# Create save directory
# ============================================================
SAVE_DIR = "/kaggle/working/s_cnn"
os.makedirs(SAVE_DIR, exist_ok=True)


# ============================================================
# 1. Density Map Generator
# ============================================================
def generate_density_map(shape, points, sigma=1.5):
    H, W = shape
    density = np.zeros((H, W), dtype=np.float32)

    for p in points:
        x, y = int(p[0]), int(p[1])
        if 0 <= x < W and 0 <= y < H:
            density[y, x] = 1.0

    return gaussian_filter(density, sigma=sigma, mode="constant")


# ============================================================
# 2. CSRNet Dataset
# ============================================================
class CSRNetDataset(Dataset):
    def __init__(self, root_dir, part='A', mode='train',
                 target_size=(512, 512), downsample_ratio=8):

        self.img_dir = os.path.join(root_dir, f"part_{part}", f"{mode}_data", "images")
        self.gt_dir = os.path.join(root_dir, f"part_{part}", f"{mode}_data", "ground-truth")

        self.img_paths = sorted(glob.glob(os.path.join(self.img_dir, "*.jpg")))
        self.target_size = target_size
        self.downsample_ratio = downsample_ratio

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        ])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")

        orig_w, orig_h = img.size
        img_resized = img.resize(self.target_size[::-1], Image.BILINEAR)

        # Load MATLAB GT
        filename = os.path.basename(img_path).replace(".jpg", "")
        mat_path = os.path.join(self.gt_dir, f"GT_{filename}.mat")

        if os.path.exists(mat_path):
            mat = loadmat(mat_path)
            points = mat["image_info"][0][0][0][0][0]
        else:
            points = np.array([])

        # Scale points
        scale_x = self.target_size[1] / orig_w
        scale_y = self.target_size[0] / orig_h
        if len(points) > 0:
            points[:,0] *= scale_x
            points[:,1] *= scale_y

        # Downsample
        out_h = self.target_size[0] // self.downsample_ratio
        out_w = self.target_size[1] // self.downsample_ratio

        points_out = points.copy()
        if len(points_out) > 0:
            points_out /= self.downsample_ratio

        density = generate_density_map((out_h, out_w), points_out, sigma=2.0)
        density_tensor = torch.from_numpy(density).unsqueeze(0)

        img_tensor = self.transform(img_resized)

        return img_tensor, density_tensor, img_path


# ============================================================
# 3. Simple CNN Model
# ============================================================
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(128,64,3,padding=1), nn.ReLU(),
            nn.Conv2d(64,32,3,padding=1), nn.ReLU(),
            nn.Conv2d(32,1,1)
        )

    def forward(self,x):
        return self.decoder(self.encoder(x))


# ============================================================
# 4. Training + Evaluation + Saving
# ============================================================
def train_csrnet():
    DATA_ROOT = "/kaggle/input/shanghaitech/ShanghaiTech"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    LR = 1e-4
    BATCH_SIZE = 4
    EPOCHS = 20

    print("Using device:", DEVICE)

    train_loader = DataLoader(CSRNetDataset(DATA_ROOT, 'A', 'train'),
                              batch_size=BATCH_SIZE, shuffle=True)

    val_loader = DataLoader(CSRNetDataset(DATA_ROOT, 'A', 'test'),
                            batch_size=1, shuffle=False)

    model = SimpleCNN().to(DEVICE)

    # Save architecture
    with open(os.path.join(SAVE_DIR, "simplecnn_architecture.txt"), "w") as f:
        f.write(str(model))

    criterion = nn.MSELoss(reduction="sum")
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    train_losses = []
    val_maes = []

    best_mae = float("inf")
    last_preds = []
    last_gts = []

    for epoch in range(EPOCHS):

        # ---------------- TRAINING ----------------
        model.train()
        total_loss = 0

        for img, target, _ in train_loader:
            img = img.to(DEVICE)
            target = target.to(DEVICE)

            pred = model(img)
            loss = criterion(pred, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        train_losses.append(total_loss)

        # ---------------- VALIDATION ----------------
        model.eval()
        mae = 0
        mse = 0
        last_preds = []
        last_gts = []

        with torch.no_grad():
            for img, target, _ in val_loader:
                img, target = img.to(DEVICE), target.to(DEVICE)
                out = model(img)

                pred_count = out.sum().item()
                gt_count = target.sum().item()

                last_preds.append(pred_count)
                last_gts.append(gt_count)

                mae += abs(pred_count - gt_count)
                mse += (pred_count - gt_count) ** 2

        mae /= len(val_loader)
        rmse = (mse / len(val_loader)) ** 0.5

        val_maes.append(mae)

        print(f"Epoch {epoch+1}/{EPOCHS} → TrainLoss={total_loss:.2f} | MAE={mae:.2f} | RMSE={rmse:.2f}")

        # Save best model
        if mae < best_mae:
            best_mae = mae
            torch.save(model.state_dict(), f"{SAVE_DIR}/best_simplecnn.pth")
            print("✔ Best model saved!")

    # ============================================================
    # SAVE METRICS CSV
    # ============================================================
    df = pd.DataFrame({
        "Epoch": list(range(1, EPOCHS+1)),
        "Train_Loss": train_losses,
        "Val_MAE": val_maes
    })
    df.to_csv(os.path.join(SAVE_DIR, "training_metrics.csv"), index=False)

    # ============================================================
    # PLOTS
    # ============================================================
    # 1️⃣ Loss & MAE curve
    plt.figure()
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_maes, label="Validation MAE")
    plt.legend()
    plt.title("Training Curve")
    plt.savefig(os.path.join(SAVE_DIR, "training_curve.png"))
    plt.close()

    # 2️⃣ GT vs Pred
    plt.figure()
    plt.scatter(last_gts, last_preds)
    plt.xlabel("GT")
    plt.ylabel("Pred")
    plt.title("GT vs Pred")
    plt.savefig(os.path.join(SAVE_DIR, "gt_vs_pred.png"))
    plt.close()

    # 3️⃣ Error Histogram
    errors = np.array(last_preds) - np.array(last_gts)
    plt.figure()
    plt.hist(errors, bins=40)
    plt.title("Error Histogram")
    plt.savefig(os.path.join(SAVE_DIR, "error_histogram.png"))
    plt.close()

    print("All training completed & plots saved.")


# ============================================================
# 5. Single Image Prediction
# ============================================================
def predict_single_image(model_path, image_path):
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    model = SimpleCNN().to(DEVICE)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.eval()

    transform = transforms.Compose([
        transforms.Resize((512,512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
    ])

    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        out = model(img_tensor)
        count = out.sum().item()

    print(f"Predicted Count = {count:.2f}")
    return count


# ============================================================
# RUN TRAINING
# ============================================================
train_csrnet()


Using device: cuda
Epoch 1/20 → TrainLoss=28135.95 | MAE=264.95 | RMSE=300.04
✔ Best model saved!
Epoch 2/20 → TrainLoss=23089.57 | MAE=182.81 | RMSE=221.09
✔ Best model saved!
Epoch 3/20 → TrainLoss=19057.91 | MAE=156.73 | RMSE=203.41
✔ Best model saved!
Epoch 4/20 → TrainLoss=17364.91 | MAE=177.63 | RMSE=224.58
Epoch 5/20 → TrainLoss=16114.19 | MAE=178.26 | RMSE=228.99
Epoch 6/20 → TrainLoss=16225.85 | MAE=151.39 | RMSE=197.93
✔ Best model saved!
Epoch 7/20 → TrainLoss=15728.47 | MAE=151.02 | RMSE=202.14
✔ Best model saved!
Epoch 8/20 → TrainLoss=14575.40 | MAE=137.41 | RMSE=179.23
✔ Best model saved!
Epoch 9/20 → TrainLoss=14445.64 | MAE=205.95 | RMSE=252.29
Epoch 10/20 → TrainLoss=13747.37 | MAE=135.12 | RMSE=185.40
✔ Best model saved!
Epoch 11/20 → TrainLoss=13189.56 | MAE=130.76 | RMSE=176.14
✔ Best model saved!
Epoch 12/20 → TrainLoss=12499.65 | MAE=254.23 | RMSE=326.97
Epoch 13/20 → TrainLoss=12298.51 | MAE=148.99 | RMSE=216.91
Epoch 14/20 → TrainLoss=11835.65 | MAE=220.25 | RM

### CSRNet Model

In [10]:
import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
from scipy.ndimage import gaussian_filter
from PIL import Image
from scipy.io import loadmat
import csv
import pandas as pd
import matplotlib.pyplot as plt
import math
from datetime import datetime


# -------------------------
# Density Map Generation
# -------------------------
def generate_density_map(shape, points, sigma=1.0):
    """
    Generate a density map for a given shape and points.
    shape: (H, W) of the output density map
    points: [(x, y), ...] coordinates on the output density map scale
    sigma: Gaussian sigma
    """
    H, W = shape
    density = np.zeros((H, W), dtype=np.float32)

    # Filter points that are out of bounds
    valid_points = []
    for p in points:
        x, y = int(p[0]), int(p[1])
        if 0 <= x < W and 0 <= y < H:
            density[y, x] = 1.0
            valid_points.append(p)

    # Apply Gaussian filter
    density = gaussian_filter(density, sigma=sigma, mode='constant', cval=0)

    return density


# -------------------------
# Dataset
# -------------------------
class CSRNetDataset(Dataset):
    def __init__(self, root_dir, part='A', mode='train', target_size=(512, 512), downsample_ratio=8):
        """
        root_dir: Path to ShanghaiTech dataset (e.g., .../ShanghaiTech)
        part: 'A' or 'B'
        mode: 'train' or 'test'
        target_size: (H, W) to resize input images to. CSRNet requires multiples of 16 usually.
        downsample_ratio: 8 for VGG16-based CSRNet
        """
        self.img_dir = os.path.join(root_dir, f"part_{part}", f"{mode}_data", "images")
        self.gt_dir = os.path.join(root_dir, f"part_{part}", f"{mode}_data", "ground-truth")

        self.img_paths = sorted(glob.glob(os.path.join(self.img_dir, "*.jpg")))
        self.target_size = target_size
        self.downsample_ratio = downsample_ratio

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]

        # Load image
        img = Image.open(img_path).convert('RGB')
        orig_w, orig_h = img.size

        # Resize image
        img_resized = img.resize(self.target_size[::-1], Image.BILINEAR) # PIL uses (W, H)

        # Load Ground Truth
        filename = os.path.basename(img_path).replace(".jpg", "")
        gt_path = os.path.join(self.gt_dir, f"GT_{filename}.mat")

        if os.path.exists(gt_path):
            mat = loadmat(gt_path)
            # ShanghaiTech mat structure: image_info -> gt_count? image_info[0][0][0][0][0]
            try:
                points = mat["image_info"][0][0][0][0][0]
            except Exception:
                # fallback: try different indexing
                try:
                    points = mat["image_info"][0][0][0][0]
                except Exception:
                    points = np.array([])
        else:
            points = np.array([])

        # Adjust points to resized image coordinates
        scale_x = self.target_size[1] / orig_w
        scale_y = self.target_size[0] / orig_h

        points_resized = points.copy()
        if len(points) > 0:
            points_resized = points_resized.astype(np.float32)
            points_resized[:, 0] *= scale_x
            points_resized[:, 1] *= scale_y

        # Generate Density Map (downsampled)
        out_h = self.target_size[0] // self.downsample_ratio
        out_w = self.target_size[1] // self.downsample_ratio

        points_map = points_resized.copy()
        if len(points) > 0:
            points_map = points_map / float(self.downsample_ratio)

        density = generate_density_map((out_h, out_w), points_map, sigma=2.0)

        # Transform image
        img_tensor = self.transform(img_resized)

        # Density to tensor
        density_tensor = torch.from_numpy(density).unsqueeze(0) # (1, H, W)

        return img_tensor, density_tensor


# -------------------------
# Model
# -------------------------
class CSRNet(nn.Module):
    def __init__(self, load_weights=True):
        super(CSRNet, self).__init__()
        self.seen = 0
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat = [512, 512, 512, 256, 128, 64]

        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat, in_channels=512, dilation=True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)

        self._initialize_weights()

        if load_weights:
            # load pretrained vgg16 and copy matching conv weights
            try:
                mod = models.vgg16(pretrained=True)
                frontend_layers = [l for l in self.frontend if isinstance(l, nn.Conv2d) or isinstance(l, nn.ReLU) or isinstance(l, nn.MaxPool2d)]
                vgg_layers = list(mod.features.children())

                i = 0
                for layer in self.frontend:
                    if isinstance(layer, nn.Conv2d):
                        # advance to next conv in vgg
                        while i < len(vgg_layers) and not isinstance(vgg_layers[i], nn.Conv2d):
                            i += 1
                        if i < len(vgg_layers) and isinstance(vgg_layers[i], nn.Conv2d):
                            try:
                                layer.weight.data.copy_(vgg_layers[i].weight.data)
                                layer.bias.data.copy_(vgg_layers[i].bias.data)
                            except Exception:
                                pass
                            i += 1
            except Exception as e:
                print("Warning: could not load pretrained VGG16 weights:", e)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg, in_channels=3, batch_norm=False, dilation=False):
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            padding = 2 if dilation else 1
            kernel_size = 3
            dilation_rate = 2 if dilation else 1
            conv2d = nn.Conv2d(in_channels, v, kernel_size=kernel_size, padding=padding, dilation=dilation_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


# -------------------------
# Utilities: saving, plotting, evaluation
# -------------------------
def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

def save_training_csv(metrics, out_path):
    df = pd.DataFrame(metrics)
    df.to_csv(out_path, index=False)

def save_model_architecture_csv(model, out_csv, dummy_input_shape=(1,3,512,512)):
    rows = []
    # add layer names and param counts
    for name, module in model.named_modules():
        if name == '':
            continue
        param_count = sum(p.numel() for p in module.parameters() if p.requires_grad)
        rows.append({'layer_name': name, 'layer_type': module.__class__.__name__, 'param_count': param_count})

    # Optional: try to get output shapes by running a dummy forward (best-effort)
    try:
        x = torch.randn(dummy_input_shape)
        x = x.to(next(model.parameters()).device)
        outputs = []
        hooks = []
        def make_hook(name):
            def hook(module, inp, out):
                try:
                    outputs.append((name, tuple(out.shape) if isinstance(out, torch.Tensor) else str(type(out))))
                except Exception:
                    outputs.append((name, 'unknown'))
            return hook
        for name, module in model.named_modules():
            if len(list(module.children())) == 0:
                hooks.append(module.register_forward_hook(make_hook(name)))
        model.eval()
        with torch.no_grad():
            _ = model(x)
        for h in hooks:
            h.remove()
        # merge output shapes into rows
        shape_map = {n:s for n,s in outputs}
        for r in rows:
            r['output_shape'] = shape_map.get(r['layer_name'], '')
    except Exception:
        # ignore shape extraction failure
        for r in rows:
            r['output_shape'] = ''

    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False)

def plot_training_loss(metrics_list, out_png):
    epochs = [m['epoch'] for m in metrics_list]
    train_loss = [m['train_loss'] for m in metrics_list]
    val_mae = [m['val_MAE'] for m in metrics_list]
    val_rmse = [m['val_RMSE'] for m in metrics_list]

    plt.figure(figsize=(8,6))
    plt.plot(epochs, train_loss, marker='o', label='train_loss')
    plt.plot(epochs, val_mae, marker='o', label='val_MAE')
    plt.plot(epochs, val_rmse, marker='o', label='val_RMSE')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('CSRNet Training Metrics')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

def evaluate_model_on_loader(model, loader, device, max_samples=None, return_all_preds=False):
    model.eval()
    preds = []
    gts = []
    img_paths = []
    with torch.no_grad():
        for i, (img, target) in enumerate(loader):
            if max_samples is not None and i >= max_samples:
                break
            img = img.to(device)
            target = target.to(device)

            output = model(img)
            pred_count = float(output.detach().cpu().sum().item())
            gt_count = float(target.detach().cpu().sum().item())

            preds.append(pred_count)
            gts.append(gt_count)
            if hasattr(loader.dataset, 'img_paths'):
                try:
                    img_paths.append(loader.dataset.img_paths[i])
                except Exception:
                    img_paths.append(None)
            else:
                img_paths.append(None)

    preds = np.array(preds)
    gts = np.array(gts)
    abs_errors = np.abs(preds - gts)
    mae = float(np.mean(abs_errors)) if len(abs_errors) > 0 else float('nan')
    rmse = float(np.sqrt(np.mean((preds - gts)**2))) if len(abs_errors) > 0 else float('nan')

    if return_all_preds:
        return {'preds': preds, 'gts': gts, 'img_paths': img_paths, 'mae': mae, 'rmse': rmse}
    else:
        return mae, rmse

def save_predictions_csv(img_paths, preds, gts, out_csv):
    rows = []
    for p, pr, gt in zip(img_paths, preds, gts):
        rows.append({'img_path': p, 'pred_count': float(pr), 'gt_count': float(gt), 'abs_error': float(abs(pr-gt))})
    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False)

def plot_gt_vs_pred(gts, preds, out_png, title="GT vs Pred"):
    plt.figure(figsize=(6,6))
    plt.scatter(gts, preds, alpha=0.6)
    mn = min(np.min(gts), np.min(preds)) if len(gts)>0 and len(preds)>0 else 0
    mx = max(np.max(gts), np.max(preds)) if len(gts)>0 and len(preds)>0 else 1
    plt.plot([mn, mx], [mn, mx], 'r--', linewidth=1)
    plt.xlabel('Ground Truth Count')
    plt.ylabel('Predicted Count')
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

def plot_residuals(gts, preds, out_png):
    residuals = preds - gts
    plt.figure(figsize=(8,5))
    plt.scatter(gts, residuals, alpha=0.6)
    plt.axhline(0, color='r', linestyle='--')
    plt.xlabel('Ground Truth Count')
    plt.ylabel('Residual (Pred - GT)')
    plt.title('Residuals vs GT')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

def plot_error_histogram(errors, out_png):
    plt.figure(figsize=(6,4))
    plt.hist(errors, bins=30)
    plt.xlabel('Absolute Error')
    plt.ylabel('Frequency')
    plt.title('Error Histogram')
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()


# -------------------------
# Training pipeline
# -------------------------
def train_csrnet():
    DATA_ROOT = "/kaggle/input/shanghaitech/ShanghaiTech"
    OUT_ROOT = "/kaggle/working/csr_net"
    ensure_dir(OUT_ROOT)

    model_dir = os.path.join(OUT_ROOT, "models")
    plots_dir = os.path.join(OUT_ROOT, "plots")
    csv_dir = os.path.join(OUT_ROOT, "csv")
    ensure_dir(model_dir); ensure_dir(plots_dir); ensure_dir(csv_dir)

    # Check if dataset exists
    if not os.path.exists(DATA_ROOT):
        print(f"Dataset not found at {DATA_ROOT}")
        return

    # Hyperparameters
    LR = 1e-5
    BATCH_SIZE = 4
    EPOCHS = 30
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    print(f"Using device: {DEVICE}")

    # Datasets
    train_ds = CSRNetDataset(DATA_ROOT, part='A', mode='train')
    val_ds = CSRNetDataset(DATA_ROOT, part='A', mode='test')

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0)

    # Model
    model = CSRNet(load_weights=True).to(DEVICE)

    # Save model architecture CSV right away (includes best-effort output shapes)
    save_model_architecture_csv(model, os.path.join(csv_dir, "model_architecture.csv"))

    criterion = nn.MSELoss(reduction='sum')
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    best_mae = float('inf')
    best_epoch = -1
    metrics_history = []

    for epoch in range(EPOCHS):
        model.train()
        train_loss_sum = 0.0
        batch_count = 0

        for i, (img, target) in enumerate(train_loader):
            img = img.to(DEVICE)
            target = target.to(DEVICE)

            output = model(img)
            loss = criterion(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_sum += float(loss.item())
            batch_count += 1

            if i % 10 == 0:
                print(f"Epoch {epoch+1}/{EPOCHS}, Step {i}/{len(train_loader)}, Loss: {loss.item():.4f}")

        train_loss_avg = train_loss_sum / max(1, batch_count)

        # Validation
        model.eval()
        mae = 0.0
        mse_sum = 0.0
        val_samples = 0

        with torch.no_grad():
            for img, target in val_loader:
                img = img.to(DEVICE)
                target = target.to(DEVICE)
                output = model(img)

                pred_count = float(output.detach().cpu().sum().item())
                gt_count = float(target.detach().cpu().sum().item())

                mae += abs(pred_count - gt_count)
                mse_sum += (pred_count - gt_count) ** 2
                val_samples += 1

        val_mae = float(mae / val_samples) if val_samples > 0 else float('nan')
        val_rmse = float(math.sqrt(mse_sum / val_samples)) if val_samples > 0 else float('nan')

        print(f"Epoch {epoch+1} Result: MAE: {val_mae:.2f}, RMSE: {val_rmse:.2f}, train_loss_avg: {train_loss_avg:.4f}")

        metrics_history.append({'epoch': epoch+1, 'train_loss': train_loss_avg, 'val_MAE': val_mae, 'val_RMSE': val_rmse})

        # Save training CSV each epoch (overwrites but keeps latest history)
        save_training_csv(metrics_history, os.path.join(csv_dir, "csr_training_metrics.csv"))

        # Save best model (by MAE)
        if val_mae < best_mae:
            best_mae = val_mae
            best_epoch = epoch + 1
            best_model_path = os.path.join(model_dir, "best_csrnet_model.pth")
            torch.save(model.state_dict(), best_model_path)
            pd.DataFrame([{'best_epoch': best_epoch, 'best_MAE': best_mae, 'best_RMSE': val_rmse, 'saved_at': datetime.now().isoformat()}]).to_csv(os.path.join(csv_dir, "best_model_metrics.csv"), index=False)
            print(f"Saved best model at epoch {best_epoch} with MAE {best_mae:.4f}")

    # End training: save final full model
    final_model_path = os.path.join(model_dir, "final_csrnet_model.pth")
    torch.save(model.state_dict(), final_model_path)
    print("Saved final model:", final_model_path)

    # Plot training metrics
    plot_training_loss(metrics_history, os.path.join(plots_dir, "csr_training_plot.png"))

    # Load best model for further evaluation & predictions
    best_model_file = os.path.join(model_dir, "best_csrnet_model.pth")
    if not os.path.exists(best_model_file):
        print("Best model file not found, using final model for evaluation")
        best_model_file = final_model_path

    best_model = CSRNet(load_weights=False).to(DEVICE)
    best_model.load_state_dict(torch.load(best_model_file, map_location=DEVICE))
    best_model.eval()

    # Evaluate best model on full validation set
    print("Evaluating best model on full validation set...")
    full_val_eval = evaluate_model_on_loader(best_model, val_loader, DEVICE, max_samples=None, return_all_preds=True)
    final_metrics_df = pd.DataFrame([{'MAE': full_val_eval['mae'], 'RMSE': full_val_eval['rmse']}])
    final_metrics_df.to_csv(os.path.join(csv_dir, "final_error_matrix_best_model.csv"), index=False)

    # Save predictions on samples for plots
    SAMPLE_N = 200
    print(f"Generating predictions on up to {SAMPLE_N} validation samples and {SAMPLE_N} train samples for plots...")
    val_sample_eval = evaluate_model_on_loader(best_model, val_loader, DEVICE, max_samples=SAMPLE_N, return_all_preds=True)
    train_sample_eval = evaluate_model_on_loader(best_model, train_loader, DEVICE, max_samples=SAMPLE_N, return_all_preds=True)

    sample_preds = val_sample_eval['preds']
    sample_gts = val_sample_eval['gts']
    sample_paths = val_sample_eval['img_paths']

    save_predictions_csv(sample_paths, sample_preds, sample_gts, os.path.join(csv_dir, "predictions_val_sample.csv"))

    # Plots: GT vs Pred for sample
    if len(sample_gts) > 0:
        plot_gt_vs_pred(sample_gts, sample_preds, os.path.join(plots_dir, "csr_gt_vs_pred.png"), title="GT vs Pred (validation sample)")
        n50 = min(50, len(sample_gts))
        plot_gt_vs_pred(sample_gts[:n50], sample_preds[:n50], os.path.join(plots_dir, "csr_gt_vs_pred_first50.png"), title="GT vs Pred (first 50 val samples)")
        plot_residuals(sample_gts, sample_preds, os.path.join(plots_dir, "csr_residual_plot.png"))
        plot_error_histogram(np.abs(sample_preds - sample_gts), os.path.join(plots_dir, "csr_error_histogram.png"))

    # Combined CSV for train and val sample comparison
    combined_rows = []
    for split_name, e in [('val', val_sample_eval), ('train', train_sample_eval)]:
        for p, pr, gt in zip(e['img_paths'], e['preds'], e['gts']):
            combined_rows.append({'split': split_name, 'img_path': p, 'pred_count': float(pr), 'gt_count': float(gt), 'abs_error': float(abs(pr-gt))})
    pd.DataFrame(combined_rows).to_csv(os.path.join(csv_dir, "predictions_combined_sample.csv"), index=False)

    print("All artifacts saved under:", OUT_ROOT)
    print(" - Models: ", model_dir)
    print(" - CSVs: ", csv_dir)
    print(" - Plots: ", plots_dir)
    print("Training finished. Best epoch:", best_epoch, "Best MAE:", best_mae)


if __name__ == "__main__":
    train_csrnet()


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 223MB/s]  


Epoch 1/30, Step 0/75, Loss: 522.3301
Epoch 1/30, Step 10/75, Loss: 496.6983
Epoch 1/30, Step 20/75, Loss: 217.6971
Epoch 1/30, Step 30/75, Loss: 766.0208
Epoch 1/30, Step 40/75, Loss: 158.6265
Epoch 1/30, Step 50/75, Loss: 125.3414
Epoch 1/30, Step 60/75, Loss: 90.4042
Epoch 1/30, Step 70/75, Loss: 115.6695
Epoch 1 Result: MAE: 104.38, RMSE: 147.70, train_loss_avg: 288.5111
Saved best model at epoch 1 with MAE 104.3764
Epoch 2/30, Step 0/75, Loss: 16.3905
Epoch 2/30, Step 10/75, Loss: 82.7026
Epoch 2/30, Step 20/75, Loss: 45.7523
Epoch 2/30, Step 30/75, Loss: 311.2673
Epoch 2/30, Step 40/75, Loss: 49.6816
Epoch 2/30, Step 50/75, Loss: 71.5258
Epoch 2/30, Step 60/75, Loss: 107.3979
Epoch 2/30, Step 70/75, Loss: 115.6520
Epoch 2 Result: MAE: 80.31, RMSE: 111.06, train_loss_avg: 83.6308
Saved best model at epoch 2 with MAE 80.3089
Epoch 3/30, Step 0/75, Loss: 53.8095
Epoch 3/30, Step 10/75, Loss: 58.9959
Epoch 3/30, Step 20/75, Loss: 61.3067
Epoch 3/30, Step 30/75, Loss: 28.2730
Epoch 3/

### MobileNet-CSRNet Model

In [1]:
import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
from scipy.ndimage import gaussian_filter
from PIL import Image
from scipy.io import loadmat
import pandas as pd
import matplotlib.pyplot as plt
import math
from datetime import datetime


# -------------------------
# Density Map Generation
# -------------------------
def generate_density_map(shape, points, sigma=1.0):
    """
    Generate a density map for a given shape and points.
    shape: (H, W) of the output density map
    points: [(x, y), ...] coordinates on the output density map scale
    sigma: Gaussian sigma
    """
    H, W = shape
    density = np.zeros((H, W), dtype=np.float32)

    # Filter points that are out of bounds
    for p in points:
        x, y = int(p[0]), int(p[1])
        if 0 <= x < W and 0 <= y < H:
            density[y, x] += 1.0

    density = gaussian_filter(density, sigma=sigma, mode='constant', cval=0)
    return density


# -------------------------
# Dataset
# -------------------------
class CSRNetDataset(Dataset):
    def __init__(self, root_dir, part='A', mode='train', target_size=(512, 512), downsample_ratio=8):
        self.img_dir = os.path.join(root_dir, f"part_{part}", f"{mode}_data", "images")
        self.gt_dir = os.path.join(root_dir, f"part_{part}", f"{mode}_data", "ground-truth")

        self.img_paths = sorted(glob.glob(os.path.join(self.img_dir, "*.jpg")))
        self.target_size = target_size
        self.downsample_ratio = downsample_ratio

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')
        orig_w, orig_h = img.size

        img_resized = img.resize(self.target_size[::-1], Image.BILINEAR)

        filename = os.path.basename(img_path).replace('.jpg', '')
        gt_path = os.path.join(self.gt_dir, f"GT_{filename}.mat")

        if os.path.exists(gt_path):
            mat = loadmat(gt_path)
            try:
                points = mat['image_info'][0][0][0][0][0]
            except Exception:
                points = np.array([])
        else:
            points = np.array([])

        scale_x = self.target_size[1] / float(orig_w)
        scale_y = self.target_size[0] / float(orig_h)

        points_resized = points.copy()
        if len(points_resized) > 0:
            points_resized = points_resized.astype(np.float32)
            points_resized[:, 0] *= scale_x
            points_resized[:, 1] *= scale_y

        out_h = self.target_size[0] // self.downsample_ratio
        out_w = self.target_size[1] // self.downsample_ratio

        points_map = points_resized.copy()
        if len(points_map) > 0:
            points_map = points_map / float(self.downsample_ratio)

        density = generate_density_map((out_h, out_w), points_map, sigma=2.0)

        img_tensor = self.transform(img_resized)
        density_tensor = torch.from_numpy(density).unsqueeze(0)

        return img_tensor, density_tensor


# -------------------------
# MobileNet-based CSRNet model
# -------------------------
class MobileNetCSRNet(nn.Module):
    def __init__(self, load_weights=True):
        super(MobileNetCSRNet, self).__init__()
        weights = models.MobileNet_V2_Weights.DEFAULT if load_weights else None
        mn = models.mobilenet_v2(weights=weights)

        # frontend: first 7 feature layers give roughly 1/8 spatial scale
        self.frontend = nn.Sequential(*list(mn.features.children())[:7])

        # backend: projection to higher channels then dilated convs
        self.backend = nn.Sequential(
            nn.Conv2d(32, 512, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(inplace=True)
        )

        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        self._initialize_weights()

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x

    def _initialize_weights(self):
        for m in self.backend.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        nn.init.normal_(self.output_layer.weight, std=0.01)
        nn.init.constant_(self.output_layer.bias, 0)


# -------------------------
# Utilities: saving, plotting, evaluation
# -------------------------
def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def save_training_csv(metrics, out_path):
    df = pd.DataFrame(metrics)
    df.to_csv(out_path, index=False)


def save_model_architecture_csv(model, out_csv, dummy_input_shape=(1,3,512,512), device='cpu'):
    rows = []
    for name, module in model.named_modules():
        if name == '':
            continue
        param_count = sum(p.numel() for p in module.parameters() if p.requires_grad)
        rows.append({'layer_name': name, 'layer_type': module.__class__.__name__, 'param_count': param_count})

    # Try to get output shapes using hooks
    try:
        model = model.to(device)
        x = torch.randn(dummy_input_shape).to(device)
        outputs = []
        hooks = []
        def make_hook(name):
            def hook(module, inp, out):
                try:
                    if isinstance(out, torch.Tensor):
                        outputs.append((name, tuple(out.shape)))
                    else:
                        outputs.append((name, str(type(out))))
                except Exception:
                    outputs.append((name, 'unknown'))
            return hook

        for name, module in model.named_modules():
            if len(list(module.children())) == 0:
                hooks.append(module.register_forward_hook(make_hook(name)))

        model.eval()
        with torch.no_grad():
            _ = model(x)
        for h in hooks:
            h.remove()

        shape_map = {n: s for n, s in outputs}
        for r in rows:
            r['output_shape'] = shape_map.get(r['layer_name'], '')
    except Exception:
        for r in rows:
            r['output_shape'] = ''

    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False)


def plot_training_loss(metrics_list, out_png):
    epochs = [m['epoch'] for m in metrics_list]
    train_loss = [m['train_loss'] for m in metrics_list]
    val_mae = [m['val_MAE'] for m in metrics_list]
    val_rmse = [m['val_RMSE'] for m in metrics_list]

    plt.figure(figsize=(8,6))
    plt.plot(epochs, train_loss, marker='o', label='train_loss')
    plt.plot(epochs, val_mae, marker='o', label='val_MAE')
    plt.plot(epochs, val_rmse, marker='o', label='val_RMSE')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('Mobile-CSRNet Training Metrics')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()


def evaluate_model_on_loader(model, loader, device, max_samples=None, return_all_preds=False):
    model.eval()
    preds = []
    gts = []
    img_paths = []
    with torch.no_grad():
        for i, (img, target) in enumerate(loader):
            if max_samples is not None and i >= max_samples:
                break
            img = img.to(device)
            target = target.to(device)

            output = model(img)
            pred_count = float(output.detach().cpu().sum().item())
            gt_count = float(target.detach().cpu().sum().item())

            preds.append(pred_count)
            gts.append(gt_count)
            if hasattr(loader.dataset, 'img_paths'):
                try:
                    img_paths.append(loader.dataset.img_paths[i])
                except Exception:
                    img_paths.append(None)
            else:
                img_paths.append(None)

    preds = np.array(preds)
    gts = np.array(gts)
    abs_errors = np.abs(preds - gts)
    mae = float(np.mean(abs_errors)) if len(abs_errors) > 0 else float('nan')
    rmse = float(np.sqrt(np.mean((preds - gts)**2))) if len(abs_errors) > 0 else float('nan')

    if return_all_preds:
        return {'preds': preds, 'gts': gts, 'img_paths': img_paths, 'mae': mae, 'rmse': rmse}
    else:
        return mae, rmse


def save_predictions_csv(img_paths, preds, gts, out_csv):
    rows = []
    for p, pr, gt in zip(img_paths, preds, gts):
        rows.append({'img_path': p, 'pred_count': float(pr), 'gt_count': float(gt), 'abs_error': float(abs(pr-gt))})
    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False)


def plot_gt_vs_pred(gts, preds, out_png, title="GT vs Pred"):
    plt.figure(figsize=(6,6))
    plt.scatter(gts, preds, alpha=0.6)
    if len(gts)>0 and len(preds)>0:
        mn = min(np.min(gts), np.min(preds))
        mx = max(np.max(gts), np.max(preds))
    else:
        mn, mx = 0, 1
    plt.plot([mn, mx], [mn, mx], 'r--', linewidth=1)
    plt.xlabel('Ground Truth Count')
    plt.ylabel('Predicted Count')
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()


def plot_residuals(gts, preds, out_png):
    residuals = preds - gts
    plt.figure(figsize=(8,5))
    plt.scatter(gts, residuals, alpha=0.6)
    plt.axhline(0, color='r', linestyle='--')
    plt.xlabel('Ground Truth Count')
    plt.ylabel('Residual (Pred - GT)')
    plt.title('Residuals vs GT')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()


def plot_error_histogram(errors, out_png):
    plt.figure(figsize=(6,4))
    plt.hist(errors, bins=30)
    plt.xlabel('Absolute Error')
    plt.ylabel('Frequency')
    plt.title('Error Histogram')
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()


# -------------------------
# Training pipeline for MobileNetCSRNet
# -------------------------

def train_mobile_csrnet():
    DATA_ROOT = "/kaggle/input/shanghaitech/ShanghaiTech"
    OUT_ROOT = "/kaggle/working/mobile_csrnet"
    ensure_dir(OUT_ROOT)

    model_dir = os.path.join(OUT_ROOT, "models")
    plots_dir = os.path.join(OUT_ROOT, "plots")
    csv_dir = os.path.join(OUT_ROOT, "csv")
    vis_dir = os.path.join(OUT_ROOT, "visualizations")
    ensure_dir(model_dir); ensure_dir(plots_dir); ensure_dir(csv_dir); ensure_dir(vis_dir)

    if not os.path.exists(DATA_ROOT):
        print(f"Dataset not found at {DATA_ROOT}")
        return

    LR = 1e-5
    BATCH_SIZE = 4
    EPOCHS = 30
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    print(f"Using device: {DEVICE}")

    train_ds = CSRNetDataset(DATA_ROOT, part='A', mode='train')
    val_ds = CSRNetDataset(DATA_ROOT, part='A', mode='test')

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0)

    model = MobileNetCSRNet(load_weights=True).to(DEVICE)

    # Save model architecture (with shapes if possible)
    save_model_architecture_csv(model, os.path.join(csv_dir, "model_architecture.csv"), device=DEVICE)

    criterion = nn.MSELoss(reduction='sum')
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    best_mae = float('inf')
    best_epoch = -1
    metrics_history = []

    for epoch in range(EPOCHS):
        model.train()
        train_loss_sum = 0.0
        batch_count = 0

        for i, (img, target) in enumerate(train_loader):
            img = img.to(DEVICE)
            target = target.to(DEVICE)

            output = model(img)
            loss = criterion(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_sum += float(loss.item())
            batch_count += 1

            if i % 10 == 0:
                print(f"Epoch {epoch+1}/{EPOCHS}, Step {i}/{len(train_loader)}, Loss: {loss.item():.4f}")

        train_loss_avg = train_loss_sum / max(1, batch_count)

        # Validation
        model.eval()
        mae = 0.0
        mse_sum = 0.0
        val_samples = 0

        with torch.no_grad():
            for img, target in val_loader:
                img = img.to(DEVICE)
                target = target.to(DEVICE)
                output = model(img)

                pred_count = float(output.detach().cpu().sum().item())
                gt_count = float(target.detach().cpu().sum().item())

                mae += abs(pred_count - gt_count)
                mse_sum += (pred_count - gt_count) ** 2
                val_samples += 1

        val_mae = float(mae / val_samples) if val_samples > 0 else float('nan')
        val_rmse = float(math.sqrt(mse_sum / val_samples)) if val_samples > 0 else float('nan')

        print(f"Epoch {epoch+1} Result: MAE: {val_mae:.2f}, RMSE: {val_rmse:.2f}, train_loss_avg: {train_loss_avg:.4f}")

        metrics_history.append({'epoch': epoch+1, 'train_loss': train_loss_avg, 'val_MAE': val_mae, 'val_RMSE': val_rmse})

        # Save training metrics CSV
        save_training_csv(metrics_history, os.path.join(csv_dir, "mobile_csr_training_metrics.csv"))

        # Save best model
        if val_mae < best_mae:
            best_mae = val_mae
            best_epoch = epoch + 1
            best_model_path = os.path.join(model_dir, "best_mobile_csrnet_model.pth")
            torch.save(model.state_dict(), best_model_path)
            pd.DataFrame([{'best_epoch': best_epoch, 'best_MAE': best_mae, 'best_RMSE': val_rmse, 'saved_at': datetime.now().isoformat()}]).to_csv(os.path.join(csv_dir, "best_model_metrics.csv"), index=False)
            print(f"Saved best model at epoch {best_epoch} with MAE {best_mae:.4f}")

    # Save final model
    final_model_path = os.path.join(model_dir, "final_mobile_csrnet_model.pth")
    torch.save(model.state_dict(), final_model_path)
    print("Saved final model:", final_model_path)

    # Plot training metrics
    plot_training_loss(metrics_history, os.path.join(plots_dir, "mobile_csr_training_plot.png"))

    # Load best model for evaluation
    best_model_file = os.path.join(model_dir, "best_mobile_csrnet_model.pth")
    if not os.path.exists(best_model_file):
        print("Best model not found, using final model for evaluation")
        best_model_file = final_model_path

    best_model = MobileNetCSRNet(load_weights=False).to(DEVICE)
    best_model.load_state_dict(torch.load(best_model_file, map_location=DEVICE))
    best_model.eval()

    # Evaluate on full validation set
    print("Evaluating best model on full validation set...")
    full_val_eval = evaluate_model_on_loader(best_model, val_loader, DEVICE, max_samples=None, return_all_preds=True)
    final_metrics_df = pd.DataFrame([{'MAE': full_val_eval['mae'], 'RMSE': full_val_eval['rmse']}])
    final_metrics_df.to_csv(os.path.join(csv_dir, "final_error_matrix_best_model.csv"), index=False)

    # Sample predictions for plots
    SAMPLE_N = 200
    print(f"Generating predictions on up to {SAMPLE_N} validation samples and {SAMPLE_N} train samples for plots...")
    val_sample_eval = evaluate_model_on_loader(best_model, val_loader, DEVICE, max_samples=SAMPLE_N, return_all_preds=True)
    train_sample_eval = evaluate_model_on_loader(best_model, train_loader, DEVICE, max_samples=SAMPLE_N, return_all_preds=True)

    sample_preds = val_sample_eval['preds']
    sample_gts = val_sample_eval['gts']
    sample_paths = val_sample_eval['img_paths']

    save_predictions_csv(sample_paths, sample_preds, sample_gts, os.path.join(csv_dir, "predictions_val_sample.csv"))

    if len(sample_gts) > 0:
        plot_gt_vs_pred(sample_gts, sample_preds, os.path.join(plots_dir, "mobile_csr_gt_vs_pred.png"), title="GT vs Pred (validation sample)")
        n50 = min(50, len(sample_gts))
        plot_gt_vs_pred(sample_gts[:n50], sample_preds[:n50], os.path.join(plots_dir, "mobile_csr_gt_vs_pred_first50.png"), title="GT vs Pred (first 50 val samples)")
        plot_residuals(sample_gts, sample_preds, os.path.join(plots_dir, "mobile_csr_residual_plot.png"))
        plot_error_histogram(np.abs(sample_preds - sample_gts), os.path.join(plots_dir, "mobile_csr_error_histogram.png"))

    combined_rows = []
    for split_name, e in [('val', val_sample_eval), ('train', train_sample_eval)]:
        for p, pr, gt in zip(e['img_paths'], e['preds'], e['gts']):
            combined_rows.append({'split': split_name, 'img_path': p, 'pred_count': float(pr), 'gt_count': float(gt), 'abs_error': float(abs(pr-gt))})
    pd.DataFrame(combined_rows).to_csv(os.path.join(csv_dir, "predictions_combined_sample.csv"), index=False)

    print("All artifacts saved under:", OUT_ROOT)
    print(" - Models: ", model_dir)
    print(" - CSVs: ", csv_dir)
    print(" - Plots: ", plots_dir)
    print("Training finished. Best epoch:", best_epoch, "Best MAE:", best_mae)


if __name__ == '__main__':
    train_mobile_csrnet()


Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


Using device: cuda


100%|██████████| 13.6M/13.6M [00:00<00:00, 121MB/s]


Epoch 1/30, Step 0/75, Loss: 592.9390
Epoch 1/30, Step 10/75, Loss: 253.5897
Epoch 1/30, Step 20/75, Loss: 196.6965
Epoch 1/30, Step 30/75, Loss: 4026.4165
Epoch 1/30, Step 40/75, Loss: 528.7634
Epoch 1/30, Step 50/75, Loss: 386.6310
Epoch 1/30, Step 60/75, Loss: 441.1340
Epoch 1/30, Step 70/75, Loss: 495.1260
Epoch 1 Result: MAE: 244.17, RMSE: 307.36, train_loss_avg: 1176.4197
Saved best model at epoch 1 with MAE 244.1733
Epoch 2/30, Step 0/75, Loss: 271.1635
Epoch 2/30, Step 10/75, Loss: 266.4446
Epoch 2/30, Step 20/75, Loss: 908.7999
Epoch 2/30, Step 30/75, Loss: 1062.1809
Epoch 2/30, Step 40/75, Loss: 2343.9758
Epoch 2/30, Step 50/75, Loss: 88.3579
Epoch 2/30, Step 60/75, Loss: 72.7854
Epoch 2/30, Step 70/75, Loss: 295.0532
Epoch 2 Result: MAE: 214.65, RMSE: 305.78, train_loss_avg: 801.8534
Saved best model at epoch 2 with MAE 214.6466
Epoch 3/30, Step 0/75, Loss: 152.4827
Epoch 3/30, Step 10/75, Loss: 53.5415
Epoch 3/30, Step 20/75, Loss: 644.6526
Epoch 3/30, Step 30/75, Loss: 362